In [5]:
!pip install qiskit
!pip install qiskit qiskit-aer
!pip install qiskit qiskit-aer --force-reinstall


  Using cached qiskit-1.4.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached qiskit_aer-0.16.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.2 kB)
  Using cached rustworkx-0.16.0-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 3.9 MB/s eta 0:00:00
  Using cached dill-0.3.9-py3-none-any.whl.metadata (10 kB)
  Using cached stevedore-5.4.1-py3-none-any.whl.metadata (2.3 kB)
  Using cached symengine-0.13.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (1.2 kB)
  Using cached pbr-6.1.1-py2.py3-none-any.whl.metadata (3.4 kB)
Using cached qiskit-1.4.1-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.7 MB)
Using cached qiskit_aer-0.16.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
Using cached dill-0.3.9-py3-non

In [ ]:
import numpy as np
from qiskit import QuantumCircuit, Aer, execute

num_bits = 20 #length of the key set here to 20

alice_data = np.random.randint(2, size=num_bits) #random generated binary string of 0 and 1  that repsents alice's msg
alice_bases = np.random.randint(2, size=num_bits) #Random sequence of 0s and 1s. Represents Alice’s choice of measurement basis (0: Standard (Z) basis, 1: Hadamard (X) basis).
bob_bases = np.random.randint(2, size=num_bits)
eve_bases = np.random.randint(2, size=num_bits) #eavesdropper

bob_data = []  #these lists store the bits measured by Bob and Eve.
eve_data = []

def encode_and_measure(bit_value, sender_basis, receiver_basis):
    circuit = QuantumCircuit(1, 1)
    if bit_value == 1: # converts |0> to |1> if the bit is 1
        circuit.x(0)
    if sender_basis == 1:
        circuit.h(0)
    if receiver_basis == 1:
        circuit.h(0)
    circuit.measure(0, 0)
    return circuit

simulator = Aer.get_backend('qasm_simulator')

for i in range(num_bits):
    eve_circuit = encode_and_measure(alice_data[i], alice_bases[i], eve_bases[i])

    #Alice sends a qubit using her chosen basis,Eve intercepts and measures the qubit using her own random basis,the result is stored in eve_data

    eve_result = execute(eve_circuit, simulator, shots=1).result().get_counts()
    eve_measured = int(list(eve_result.keys())[0])
    eve_data.append(eve_measured)

     #Bob receives the qubit (which may have been altered by Eve),Bob measures the qubit using his own random basis,the result is stored in bob_data

    bob_circuit = encode_and_measure(eve_measured, eve_bases[i], bob_bases[i])
    bob_result = execute(bob_circuit, simulator, shots=1).result().get_counts()
    bob_measured = int(list(bob_result.keys())[0])
    bob_data.append(bob_measured)

bob_data = np.array(bob_data)
eve_data = np.array(eve_data)

matched_positions = [] #list that stores the indexes at which alice and bob used the same bases
alice_key = []
bob_key = []
eve_key = []

for i in range(num_bits):
    if alice_bases[i] == bob_bases[i]:
        matched_positions.append(True)
        alice_key.append(alice_data[i])
        bob_key.append(bob_data[i])
        eve_key.append(eve_data[i])
    else:
        matched_positions.append(False)
 #If Alice and Bob used the same basis, their measurement should match, and the bit is added to their final secret key otherwise the bit is discarded

def calculate_parity(bit_sequence):
    return sum(bit_sequence) % 2

alice_parity = calculate_parity(alice_key)
bob_parity = calculate_parity(bob_key)

# if the sum of the bits of alice and bob have different parity that means Eve has intercepted the channel and collapsed the system to some random quantum state.
print("\nFinal Keys:")
print("Alice's Key:", alice_key)
print("Bob's Key:", bob_key)
print("Eve's Key:", eve_key)

if alice_parity == bob_parity:
    print("\nKey Exchange Successful: No interference detected.")
else:
    print("\nWarning: Possible eavesdropping detected!")



ModuleNotFoundError: No module named 'qiskit.providers.aer'